# Greed Simulations

Creating simulations to test game-wide strategies (first to 100 points).

In [5]:
# ----- import libraries -----

import random
import matplotlib

In [6]:
# ----- helper functions -----
def sumList(sum_list: list)->int:
    
    """
    
    sums the values of a list.

    Parameters:
    -----------

    sum_list: list
        list of integers to find the sum of.

    Returns:
    --------

    sum: int
        sum of the values in the list.

    """
    
    sum = 0
    for value in sum_list:
        sum += value
    
    return sum

def rollDie()->int:
    
    """
    
    'rolls' a die and returns the result.

    Parameters:
    -----------

    None

    Returns:
    --------

    result: int
        integer representing the value of the roll of the die.

    """

    return random.randInt(1, 6)

In [3]:
# ----- create strategy class -----
class strategy():

    def __init__(self, name:str, decision_function:function)->None:

        #initialise the strategy class instance for a specific strategy

        self.name = name #string representing the name of the strategy
        self.decision_function = decision_function #function for the strategy, taking in a turn list and returning True or False depending on accept score/roll again
        
        self.winning_score = 0 #variable representing the score needed to win
        self.current_score = 0 #variable representing the current score (in the bank) the strategy has

    def getDecision(self, current_rolls: list)->bool:
        
        """
        given parameters a player can take into account, make a decision as to whether the strategy should accept the current score or roll again.

        Parameters:
        -----------

        current_rolls: list
            list of integers representing the current roll-set.

        Returns:
        --------

        decision: bool
            boolean representing the strategy's decision. True if accept, False if roll again.

        """

        return self.decision_function(self.winning_score, self.current_score, current_rolls)


    

In [7]:
# ----- create functions to test strategies against one another -----

def runSimulation(strategy_1:strategy, strategy_2:strategy, winning_score:int)->list: 
    
    """
    runs a single simulation of a game between two strategies.

    Parameters:
    -----------

    strategy_1: strategy object
        strategy object for the first strategy in the simulation.

    strategy_2: strategy object
        strategy object for the second strategy in the simulation.

    winning_score: int
        integer representing the score needed to win a game.

    Returns:
    --------

    results: list
        tuple containing the results of the simulation. formatted as follows:
        [winner:string, margin:int]

    """

    #ensure both strategies have their scores reset, and the winning score updated

    strategy_1.winning_score = winning_score
    strategy_1.current_score = 0

    strategy_2.winning_score = winning_score
    strategy_2.current_score = 0
    

    #loop the turns until one strategy gets enough points to win
    while True:

        #strategy_1 turn
        current_moves = []
        while (strategy_1.current_score + sumList(current_moves) < winning_score):

            roll = rollDie()

            if roll == 1:
                current_moves = [] #reset the moves so that the strategy has no points added
                break

            else:
                current_moves.append(roll) #add the result of the roll to the moves list
                if not strategy_1.getDecision(current_moves):
                    break

        strategy_1.current_score = strategy_1.current_score + sumList(current_moves) #update the score of strategy_1

        #check that strategy_1 has not won yet; if so, game is over
        if strategy_1.current_score >= winning_score:
            break

        #strategy_2 turn
        current_moves = []
        while (strategy_2.current_score + sumList(current_moves) < winning_score):

            roll = rollDie()

            if roll == 1:
                current_moves = [] #reset the moves so that the strategy has no points added
                break

            else:
                current_moves.append(roll) #add the result of the roll to the moves list
                if not strategy_2.getDecision(current_moves):
                    break

        strategy_2.current_score = strategy_2.current_score + sumList(current_moves) #update the score of strategy_2

        #check that strategy_2 has not won yet; if so, game is over
        if strategy_2.current_score >= winning_score:
            break

    
    #someone has now won, return results accordingly
    winner = ""
    margin = 0

    if strategy_1.current_score >= winning_score:

        winner = strategy_1.name
        margin = 100 - strategy_2.current_score

    else:

        winner = strategy_2.name
        margin = 100 - strategy_1.current_score

    return [winner, margin]

def runSimulationBlock(strategy_1:strategy, strategy_2:strategy, winning_score:int, games:int)->dict:
    
    """

    runs a set of simulations between two strategies.

    Parameters:
    -----------

    strategy_1: strategy object
        strategy object for the first strategy in the simulation.

    strategy_2: strategy object
        strategy object for the second strategy in the simulation.

    winning_score: int
        integer representing the score needed to win a game.

    games: int
        integer representing the number of games to simulate.

    Returns:
    --------

    results: dict
        dictionary of results. formatted as follows:
        {
            "win_count" : [strategy_1_wins, strategy_2_wins],
            "win_margins" : [strategy_1_win_margin_list, strategy_2_win_margin_list]
            "average_margins" : [strategy_1_average_margin, strategy_2_average_margin]
        }

    """

    print(f"running simulations for {strategy_1.name} vs. {strategy_2.name}: \n \n")

    # ----- run the games -----

    results_list = []

    for i in range(0, games):
    
        results = runSimulation(strategy_1, strategy_2, winning_score)
        results_list.append(results)

        print(f"     game {i} of {games}: {results}\n")

    # ----- create the results dictionary -----

    results = {}
    
    #assemble win count, margins

    strategy_1_wins = 0
    strategy_2_wins = 0
    strategy_1_margins = []
    strategy_2_margins = []

    for result in results_list:

        if result[0] == strategy_1.name:

            strategy_1_wins += 1
            strategy_1_margins.append(result[1])
        
        else:

            strategy_2_wins += 1
            strategy_2_margins.append(result[1])

    results["win_count"] = [strategy_1_wins, strategy_2_wins]
    results["win_margins"] = [strategy_1_margins, strategy_2_margins]

    #assemble average win margins

    strategy_1_average = (sumList(results["win_margins"][0]))/(len(results["win_margins"][0]))
    strategy_2_average = (sumList(results["win_margins"][1]))/(len(results["win_margins"][1]))

    results["average_margins"] = [strategy_1_average, strategy_2_average]

    # ----- return final results dict ----- 

    return results


In [ ]:
strategies = []

# ----- test all strategies against one another -----
def testStrategies(strategies:list)->dict:
    pass